In [1]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd

from pathlib import Path

from rasterstats import zonal_stats
import rasterio
from rasterio.merge import merge
from rasterio.plot import show
import glob, os
# run for jupyter notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

### Folder Structure

In [12]:
#%% Read in data
rice_root = '/home/apoorval/Research/GeoSpatial/India_Forests/'
dbox_root = '/home/alal/res/India_Forests/'
root = Path(dbox_root)
code = root / 'Code'
data = root / 'Data'
spatial = data/'Spatial'

In [3]:
%%time
vil_treat = gpd.read_file(data/"Spatial/Processed/Village_stack_treatment.gpkg")
vil_treat.head()

CPU times: user 43.5 s, sys: 11.8 s, total: 55.3 s
Wall time: 54.6 s


,JHAR_ID,NAME,SUB_DIST,DISTRICT,STATE_UT,CODE_2011,LEVEL,TOT_HH,TOT_POP,M_POP,...,GUJAR_ID,HP_ID,MADHYA_ID,MAHARA_ID,RAJAS_ID,index_right,sch,named,nameb,geometry
0,1.0,Janumpi,Majhgaon,Pashchimi Singhbhum,Jharkhand,2036802745378466,Village,73.0,399.0,199.0,...,NaN,NaN,NaN,NaN,NaN,1622,1,pashchimi singhbhum,majhgaon,"POLYGON ((85.89696 21.98907, 85.89709 21.98904..."
1,2.0,Buruikuti,Majhgaon,Pashchimi Singhbhum,Jharkhand,2036802745378491,Village,122.0,710.0,347.0,...,NaN,NaN,NaN,NaN,NaN,1622,1,pashchimi singhbhum,majhgaon,"POLYGON ((85.83532 21.99219, 85.83662 21.99089..."
2,3.0,Benisagar,Majhgaon,Pashchimi Singhbhum,Jharkhand,2036802745378467,Village,137.0,667.0,319.0,...,NaN,NaN,NaN,NaN,NaN,1622,1,pashchimi singhbhum,majhgaon,"POLYGON ((85.88185 21.99527, 85.88362 21.99454..."
3,4.0,Tiraposi,Majhgaon,Pashchimi Singhbhum,Jharkhand,2036802745378492,Village,163.0,930.0,460.0,...,NaN,NaN,NaN,NaN,NaN,1622,1,pashchimi singhbhum,majhgaon,"POLYGON ((85.87063 22.00095, 85.87117 22.00030..."
4,5.0,Balibandh,Majhgaon,Pashchimi Singhbhum,Jharkhand,2036802745378496,Village,306.0,1558.0,753.0,...,NaN,NaN,NaN,NaN,NaN,1622,1,pashchimi singhbhum,majhgaon,"POLYGON ((85.77795 22.00883, 85.77991 22.00846..."


In [4]:
deforestation = root / 'Data/Spatial/Rasters/Hansen_GFC-2017-v1.5_lossyear_30N_080E.tif'
ex_ante       = root / 'Data/Spatial/Rasters/Hansen_GFC-2017-v1.5_treecover2000_30N_080E.tif'

with rasterio.open(deforestation) as src:
    print(src.profile)

with rasterio.open(ex_ante) as src:
    print(src.profile)

{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 40000, 'height': 40000, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.00025, 0.0, 80.0,
       0.0, -0.00025, 30.0), 'tiled': False, 'compress': 'lzw', 'interleave': 'band'}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 40000, 'height': 40000, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.00025, 0.0, 80.0,
       0.0, -0.00025, 30.0), 'tiled': False, 'compress': 'lzw', 'interleave': 'band'}


# Dry Run

## Raster Merge 

### Dry Run 

### 1 district 

In [ ]:
sim_def =  gpd.GeoDataFrame.from_features(zonal_stats(simdega, 
                    deforestation, prefix = 'deforest_',
                    stats='count', nodata=-1, categorical=True, geojson_out = True))

In [ ]:
sim_def.head()

In [ ]:
sim_def['deforest_4'].value_counts()

## Run Merge 

### 1 State 

In [ ]:
j_def =  gpd.GeoDataFrame.from_features(zonal_stats(jharkhand, deforestation, 
                    prefix = 'deforest_',
                    stats='count', nodata=-1, categorical=True, geojson_out = True))

In [ ]:
j_def.to_file(data + '/Spatial/Processed/jharkhand_village_deforestation.shp')

# Merge All 

In [5]:
rasters = root / 'Data/Spatial/Rasters/'
%ls {rasters}

Hansen_GFC-2017-v1.5_datamask_20N_070E.tif*
Hansen_GFC-2017-v1.5_datamask_20N_080E.tif*
Hansen_GFC-2017-v1.5_datamask_30N_080E.tif*
Hansen_GFC-2017-v1.5_datamask_40N_070E.tif*
Hansen_GFC-2017-v1.5_lossyear_20N_070E.tif*
Hansen_GFC-2017-v1.5_lossyear_20N_080E.tif*
Hansen_GFC-2017-v1.5_lossyear_30N_070E.tif*
Hansen_GFC-2017-v1.5_lossyear_30N_080E.tif*
Hansen_GFC-2017-v1.5_lossyear_40N_070E.tif*
Hansen_GFC-2017-v1.5_treecover2000_20N_070E.tif*
Hansen_GFC-2017-v1.5_treecover2000_20N_080E.tif*
Hansen_GFC-2017-v1.5_treecover2000_30N_070E.tif*
Hansen_GFC-2017-v1.5_treecover2000_30N_080E.tif*
Hansen_GFC-2017-v1.5_treecover2000_40N_070E.tif*
_Hansen_GFC_lossyear_all.tif
_Hansen_GFC_lossyear_all.tif.aux.xml*
_Hansen_GFC_treecover2000_all.tif
downloader.sh*
forest_rasters.txt*


In [6]:
def_master = rasters / '_Hansen_GFC_lossyear_all.tif'

In [7]:
with rasterio.open(def_master) as src:
    print(src.profile)

{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 80000, 'height': 120000, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.00025, 0.0, 70.0,
       0.0, -0.00025, 40.0), 'tiled': False, 'compress': 'lzw', 'interleave': 'band'}


In [8]:
%%time
all_def = gpd.GeoDataFrame.from_features(zonal_stats(vil_treat, def_master,   
            prefix = 'deforest_', stats='count', nodata=-1, 
            categorical=True, geojson_out=True))
all_def.crs = vil_treat.crs

CPU times: user 1h 23min 32s, sys: 8min 31s, total: 1h 32min 4s
Wall time: 1h 32min 11s


In [9]:
all_def.head()
all_def.shape

,geometry,ANDHR_ID,CHHAT_ID,CODE_2011,DISTRICT,F_AGLB,F_CULT,F_ILLT,F_L6,F_LIT,...,deforest_9,deforest_12,deforest_3,deforest_7,deforest_2,deforest_14,deforest_17,deforest_13,deforest_15,deforest_16
0,"POLYGON ((85.89696 21.98907, 85.89709 21.98904...",NaN,NaN,2036802745378466,Pashchimi Singhbhum,1.0,9.0,155.0,41.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"POLYGON ((85.83532 21.99219, 85.83662 21.99089...",NaN,NaN,2036802745378491,Pashchimi Singhbhum,47.0,10.0,264.0,68.0,99.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"POLYGON ((85.88185 21.99527, 85.88362 21.99454...",NaN,NaN,2036802745378467,Pashchimi Singhbhum,3.0,2.0,232.0,63.0,116.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"POLYGON ((85.87063 22.00095, 85.87117 22.00030...",NaN,NaN,2036802745378492,Pashchimi Singhbhum,0.0,1.0,394.0,88.0,76.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"POLYGON ((85.77795 22.00883, 85.77991 22.00846...",NaN,NaN,2036802745378496,Pashchimi Singhbhum,113.0,95.0,533.0,154.0,272.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(308503, 124)

In [13]:
%%time
all_def.to_file(spatial/'Processed/village_stack_deforestation.gpkg', driver = "GPKG")

CPU times: user 17min 11s, sys: 6.12 s, total: 17min 18s
Wall time: 17min 19s


# Ex-Ante Forest Levels 

In [14]:
rasters = spatial/ 'Rasters/'
%ls {rasters}

Hansen_GFC-2017-v1.5_datamask_20N_070E.tif*
Hansen_GFC-2017-v1.5_datamask_20N_080E.tif*
Hansen_GFC-2017-v1.5_datamask_30N_080E.tif*
Hansen_GFC-2017-v1.5_datamask_40N_070E.tif*
Hansen_GFC-2017-v1.5_lossyear_20N_070E.tif*
Hansen_GFC-2017-v1.5_lossyear_20N_080E.tif*
Hansen_GFC-2017-v1.5_lossyear_30N_070E.tif*
Hansen_GFC-2017-v1.5_lossyear_30N_080E.tif*
Hansen_GFC-2017-v1.5_lossyear_40N_070E.tif*
Hansen_GFC-2017-v1.5_treecover2000_20N_070E.tif*
Hansen_GFC-2017-v1.5_treecover2000_20N_080E.tif*
Hansen_GFC-2017-v1.5_treecover2000_30N_070E.tif*
Hansen_GFC-2017-v1.5_treecover2000_30N_080E.tif*
Hansen_GFC-2017-v1.5_treecover2000_40N_070E.tif*
_Hansen_GFC_lossyear_all.tif
_Hansen_GFC_lossyear_all.tif.aux.xml*
_Hansen_GFC_treecover2000_all.tif
downloader.sh*
forest_rasters.txt*


## Raster Merge 

In [15]:
ea_master = rasters / '_Hansen_GFC_treecover2000_all.tif'

In [16]:
with rasterio.open(ea_master) as src:
    print(src.profile)

{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 80000, 'height': 120000, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.00025, 0.0, 70.0,
       0.0, -0.00025, 40.0), 'tiled': False, 'compress': 'lzw', 'interleave': 'band'}


In [17]:
%%time
all_def = gpd.GeoDataFrame.from_features(zonal_stats(vil_treat, ea_master,   
            prefix = 'preF_', nodata=-1, geojson_out = True))
all_def.crs = vil_treat.crs

CPU times: user 2h 21min 52s, sys: 13min 40s, total: 2h 35min 32s
Wall time: 2h 35min 42s


In [18]:
all_def.head()
all_def.shape

,geometry,ANDHR_ID,CHHAT_ID,CODE_2011,DISTRICT,F_AGLB,F_CULT,F_ILLT,F_L6,F_LIT,...,T_M_CL_0_3,T_M_CL_3_6,index_right,nameb,named,sch,preF_min,preF_max,preF_mean,preF_count
0,"POLYGON ((85.89696 21.98907, 85.89709 21.98904...",NaN,NaN,2036802745378466,Pashchimi Singhbhum,1.0,9.0,155.0,41.0,45.0,...,10.0,47.0,1622,majhgaon,pashchimi singhbhum,1,0.0,12.0,0.020349,3096
1,"POLYGON ((85.83532 21.99219, 85.83662 21.99089...",NaN,NaN,2036802745378491,Pashchimi Singhbhum,47.0,10.0,264.0,68.0,99.0,...,3.0,5.0,1622,majhgaon,pashchimi singhbhum,1,0.0,32.0,0.073713,5033
2,"POLYGON ((85.88185 21.99527, 85.88362 21.99454...",NaN,NaN,2036802745378467,Pashchimi Singhbhum,3.0,2.0,232.0,63.0,116.0,...,46.0,137.0,1622,majhgaon,pashchimi singhbhum,1,0.0,28.0,0.300000,3400
3,"POLYGON ((85.87063 22.00095, 85.87117 22.00030...",NaN,NaN,2036802745378492,Pashchimi Singhbhum,0.0,1.0,394.0,88.0,76.0,...,0.0,0.0,1622,majhgaon,pashchimi singhbhum,1,0.0,14.0,0.031380,5035
4,"POLYGON ((85.77795 22.00883, 85.77991 22.00846...",NaN,NaN,2036802745378496,Pashchimi Singhbhum,113.0,95.0,533.0,154.0,272.0,...,30.0,95.0,1622,majhgaon,pashchimi singhbhum,1,0.0,32.0,0.424898,9840


(308503, 109)

In [ ]:
%%time
all_def.to_file(spatial/'Processed/village_stack_ex_ante.gpkg', driver = "GPKG")